# Importing Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom as dicom
import pandas as pd
import numpy as np
from random import randint

import os, sys, time, shutil, scipy, cv2, json, datetime
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm.notebook import tqdm

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler 
import albumentations as A
import geffnet

from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, roc_auc_score

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, optimizers

# Build a baseline fully connected model
from keras import models
from keras import layers

# Baseline Model:  Densely Connected Network

In [4]:
train = pd.read_csv('../data/train_df.csv', index_col=0)
train.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,ISIC_2637011.jpg
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,ISIC_0015719.jpg
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,ISIC_0052212.jpg
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,ISIC_0068279.jpg
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,ISIC_0074268.jpg


In [5]:
test = pd.read_csv('../data/test_df.csv', index_col=0)
test.head()

,file,age,site,lesion_id,gender,target,jpg
0,ISIC_0000000,55.0,anterior torso,NaN,female,0,ISIC_0000000.jpg
1,ISIC_0000001,30.0,anterior torso,NaN,female,0,ISIC_0000001.jpg
2,ISIC_0000002,60.0,upper extremity,NaN,female,1,ISIC_0000002.jpg
3,ISIC_0000003,30.0,upper extremity,NaN,male,0,ISIC_0000003.jpg
4,ISIC_0000004,80.0,posterior torso,NaN,male,1,ISIC_0000004.jpg


In [8]:
WORK_DIR = "../data"
label_col = "target"
img_col = "jpg"
train_folder = "../images/train"
test_folder = "../images/test"

os.listdir(WORK_DIR)

['ISIC_2019_Training_GroundTruth.csv',
 'ISIC_2019_Training_Metadata.csv',
 'ISIC_2020_Training_Duplicates.csv',
 'ISIC_2020_Training_GroundTruth_v2.csv',
 'test.csv',
 'test_df.csv',
 'train.csv',
 'train_df.csv',
 'val.csv']

In [9]:
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train_df.csv"), index_col=0)
label_names = train_labels[label_col].value_counts().index
label_map = {name:i for (i,name) in enumerate(label_names)}
inv_label_map = {v: k for k, v in label_map.items()}

train_labels['label_name'] = train_labels[label_col].copy()
train_labels[label_col] = train_labels[label_col].map(label_map)
display(train_labels.head())

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg,label_name
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,ISIC_2637011.jpg,0
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,ISIC_0015719.jpg,0
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,ISIC_0052212.jpg,0
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,ISIC_0068279.jpg,0
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,ISIC_0074268.jpg,0


In [6]:
test.head()

,file,age,site,lesion_id,gender,target,jpg
0,ISIC_0000000,55.0,anterior torso,NaN,female,0,ISIC_0000000.jpg
1,ISIC_0000001,30.0,anterior torso,NaN,female,0,ISIC_0000001.jpg
2,ISIC_0000002,60.0,upper extremity,NaN,female,1,ISIC_0000002.jpg
3,ISIC_0000003,30.0,upper extremity,NaN,male,0,ISIC_0000003.jpg
4,ISIC_0000004,80.0,posterior torso,NaN,male,1,ISIC_0000004.jpg


In [10]:
# Main parameters
BATCH_SIZE = 8
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE
EPOCHS = 4
TARGET_SIZE = 512

In [11]:
train.target = train.target.astype(str)
test.target = test.target.astype(str)

In [34]:
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.20)
train_generator = datagen.flow_from_dataframe(
    dataframe = train,
    directory = "./images/train_jpg",
    x_col = "jpg",
    y_col = "target",
    subset = "training",
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    target_size = (256,256))

valid_generator = datagen.flow_from_dataframe(
    dataframe = train,
    directory = "./images/train_jpg",
    x_col = "jpg",
    y_col = "target",
    subset = "validation",
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    target_size = (256,256))

test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory="./images/test/",
    x_col = "jpg",
    y_col = 'target',
    batch_size = 32,
    seed = 42,
    shuffle = False,
    class_mode = 'categorical',
    target_size = (256,256))

Found 26501 validated image filenames belonging to 2 classes.
Found 6625 validated image filenames belonging to 2 classes.
Found 25331 validated image filenames belonging to 2 classes.


In [35]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(valid_generator)

In [36]:
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 32
Number of testing samples: 32
Number of validation samples: 32
train_images shape: (32, 256, 256, 3)
train_labels shape: (32, 2)
test_images shape: (32, 256, 256, 3)
test_labels shape: (32, 2)
val_images shape: (32, 256, 256, 3)
val_labels shape: (32, 2)


In [37]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(32, 196608)
(32, 196608)
(32, 196608)


In [38]:
train_y = np.reshape(train_labels[:,0], (32,1))
test_y = np.reshape(test_labels[:,0], (32,1))
val_y = np.reshape(val_labels[:,0], (32,1))

In [17]:
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(196608,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [18]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 0.6604 - accuracy: 0.9688 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 77ms/step - loss: 0.1894 - accuracy: 0.9688 - val_loss: 0.0355 - val_accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 77ms/step - loss: 0.1219 - accuracy: 0.9688 - val_loss: 0.0227 - val_accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 78ms/step - loss: 0.1194 - accuracy: 0.9688 - val_loss: 0.0491 - val_accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 150ms/step - loss: 0.1254 - accuracy: 0.9688 - val_loss: 0.0115 - val_accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 75ms/step - loss: 0.1243 - accuracy: 0.9688 - val_loss: 0.0558 - val_accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 76ms/step - loss: 0.1254 - accuracy: 0.9688 - val_loss: 0.0199 - val_accuracy: 1.0000
Epoch 8/50
1/1 [============

In [19]:
results_train = model.evaluate(train_img, train_y)

1/1 [==============================] - 0s 25ms/step - loss: 0.0551 - accuracy: 0.9688


In [20]:
results_test = model.evaluate(test_img, test_y)

1/1 [==============================] - 0s 23ms/step - loss: 1.1338 - accuracy: 0.7812


In [21]:
results_train

[0.05513884872198105, 0.96875]

In [22]:
results_test

[1.1337727308273315, 0.78125]

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                3932180   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 147       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 40        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 3,932,373
Trainable params: 3,932,373
Non-trainable params: 0
_________________________________________________________________


# Build a CNN

In [24]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [25]:
history = model.fit(train_images,
                    train_y,
                    epochs=30,
                    batch_size=32,
                    validation_data=(val_images, val_y))

Epoch 1/30
1/1 [==============================] - 4s 4s/step - loss: 0.7315 - acc: 0.0312 - val_loss: 0.0177 - val_acc: 1.0000
Epoch 2/30
1/1 [==============================] - 2s 2s/step - loss: 0.1345 - acc: 0.9688 - val_loss: 0.0215 - val_acc: 1.0000
Epoch 3/30
1/1 [==============================] - 2s 2s/step - loss: 0.1317 - acc: 0.9688 - val_loss: 0.0246 - val_acc: 1.0000
Epoch 4/30
1/1 [==============================] - 2s 2s/step - loss: 0.1301 - acc: 0.9688 - val_loss: 0.0270 - val_acc: 1.0000
Epoch 5/30
1/1 [==============================] - 2s 2s/step - loss: 0.1291 - acc: 0.9688 - val_loss: 0.0290 - val_acc: 1.0000
Epoch 6/30
1/1 [==============================] - 3s 3s/step - loss: 0.1286 - acc: 0.9688 - val_loss: 0.0302 - val_acc: 1.0000
Epoch 7/30
1/1 [==============================] - 2s 2s/step - loss: 0.1283 - acc: 0.9688 - val_loss: 0.0313 - val_acc: 1.0000
Epoch 8/30
1/1 [==============================] - 2s 2s/step - loss: 0.1280 - acc: 0.9688 - val_loss: 0.0320 - 

In [26]:
results_train = model.evaluate(train_images, train_y)

1/1 [==============================] - 0s 332ms/step - loss: 0.1249 - acc: 0.9688


In [40]:
results_test = model.evaluate(test_images, test_y)

1/1 [==============================] - 1s 541ms/step - loss: 0.8336 - acc: 0.7500


In [41]:
results_train

[0.12485035508871078, 0.96875]

In [42]:
results_test

[0.8335974216461182, 0.75]